In [1]:
import sys
import boto3
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from awsglue.context import GlueContext
from awsglue.job import Job

In [3]:
sys.argv += ['--JOB_NAME', 'glue_script']
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

spark = SparkSession.builder.config('spark.serializer','org.apache.spark.serializer.KryoSerializer').getOrCreate()
sc = spark.sparkContext
glueContext = GlueContext(sc)
job = Job(glueContext)
job.init(args['JOB_NAME'], args)
sc.getConf().getAll()

[('spark.executor.extraClassPath', '/opt/aws-glue-libs/jarsv1/*'),
 ('spark.sql.warehouse.dir', 'file:/opt/jupyter/workspace/spark-warehouse'),
 ('spark.driver.extraClassPath', '/opt/aws-glue-libs/jarsv1/*'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.driver.port', '35365'),
 ('spark.app.startTime', '1634485674146'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', 'ac4f70216706'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.id', 'local-1634485675752')]

In [4]:
os.environ["TEST_S3_ENDPOINT_URL"] = "http://glue.dev.s3.local:4566"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://glue.dev.s3.local:4566")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.signing-algorithm", "S3SignerType")

In [7]:
session = boto3.session.Session(
    aws_access_key_id = 'xxx',
    aws_secret_access_key = 'xxx'
)
s3client = session.client(
    "s3",
    endpoint_url=os.environ["TEST_S3_ENDPOINT_URL"],
    use_ssl=False,
)
bucket = s3client.create_bucket(Bucket = 'test-bucket')

In [8]:
s3 = boto3.resource(
    "s3",
    endpoint_url=os.environ["TEST_S3_ENDPOINT_URL"],
    region_name="ap-northeast-1",
    use_ssl=False,
   
)
bucket_name = "test-bucket"
bucket = s3.Bucket(bucket_name)
bucket.upload_file("test_data/sample.json", "sample/sample.json")

In [10]:
p = f"s3://{bucket_name}/sample/sample.json"
df = glueContext.create_dynamic_frame.from_options(
    connection_type="s3",
    connection_options={"paths": [p]},
    format="json"
).toDF()
df.count()
df.show()

+------+
|  name|
+------+
|  taro|
|  jiro|
|saburo|
| shiro|
+------+



In [11]:
df.createOrReplaceTempView("sample")
spark.sql("show tables").show()
spark.sql("select count(*) from sample").show()
spark.sql("select * from sample").show()


ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.7.2ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.7.2ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.7.2ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.7.221/10/17 15:58:30 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/10/17 15:58:30 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/10/17 15:58:39 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
21/10/17 15:58:39 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@192.168.32.3
21/10/17 15:58:39 WARN ObjectStore: Failed to get database default, returning NoSuchObjec

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |   sample|       true|
+--------+---------+-----------+

+--------+
|count(1)|
+--------+
|       4|
+--------+

+------+
|  name|
+------+
|  taro|
|  jiro|
|saburo|
| shiro|
+------+

